In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt 
import datetime

In [2]:
from mochi import find_related_recent_trades_within_days,find_related_recent_logins_within_days
from mochi import get_multiple_feature_dicts_wihtin_days,get_transfer_result_wihtin_days
from mochi import check_nan_inf,get_feature_emptyness

from mochi import mult_type_hcc_feature

In [3]:
class Config:
    pass
config = pd.read_pickle('config.pkl')
print(dir(config))

data_path = '../../kaggleData/JD_logging/'
feature_path = '../../kaggleData/JD_logging/features/'

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'data_path', 'feature_dict', 'feature_path', 'model_features', 'result_path', 'single_module_validation_indice_set', 'trade_train_size', 'train_2_6_index']


In [4]:
login_tt = pd.read_csv(data_path+'login_tt.csv')
trade_tt = pd.read_csv(data_path+'trade_tt.csv')
trade_tt_with_recent_logins = pd.read_csv(data_path+'trade_tt_with_recent_3_log_id.csv')


login_tt['time'] = login_tt['time'].apply(lambda x : datetime.datetime.strptime(x , '%Y-%m-%d %H:%M:%S'))
trade_tt['time'] = trade_tt['time'].apply(lambda x : datetime.datetime.strptime(x , '%Y-%m-%d %H:%M:%S'))
trade_tt_with_recent_logins['time'] = trade_tt_with_recent_logins['time'].apply(lambda x : datetime.datetime.strptime(x , '%Y-%m-%d %H:%M:%S'))



In [5]:
#has trade
login_tt['has_trade'] = np.isnan(login_tt.merge(trade_tt_with_recent_logins,how = 'left',left_on = 'log_id',
right_on = 'recent_login_id_0',suffixes = ['','_trade'])['recent_login_id_0'])

login_tt_has_trade = login_tt[login_tt['has_trade']]

In [6]:
#hcc_feautre = ['ip','city','device']
def build_hcc_stats_feature(recent_login_table,recent_trade_table,hcc_feature):
    login_size = len(recent_login_table)
    
    if login_size==0:
        #TODO:return -10 for all the results
        return {}
    
    result_dict = {}
    
    recent_logins = recent_login_table.sort_values(by = 'time',ascending = False)
    last_login = recent_logins.iloc[0]
    
    hcc_target = last_login[hcc_feature]
    
    result_dict[hcc_feature+'_repeat_count'] = len(recent_login_table[recent_login_table[hcc_feature] == hcc_target])
    
    result_dict[hcc_feature+'_repeat_rate']  = result_dict[hcc_feature+'_repeat_count']*1.0/login_size
    
    result_dict[hcc_feature+'_used_count'] = len(recent_login_table[hcc_feature].unique())
    
    hcc_feature_freq = recent_login_table.groupby(hcc_feature).size()
    
    hcc_feature_rank = hcc_feature_freq.rank(method = 'min').sort_values(ascending = False)
    
    result_dict[hcc_feature+'_freq_rank'] = hcc_feature_rank[hcc_target]
        
    result_dict[hcc_feature+'_most_freq_count'] = hcc_feature_freq.max()
    
    result_dict[hcc_feature+'_most_freq_rate'] = result_dict[hcc_feature+'_most_freq_count']*1.0/login_size
        
    result_dict[hcc_feature+'_max_min_dist'] =  (result_dict[hcc_feature+'_most_freq_count'] - hcc_feature_freq.min())*1.0/\
                                                 result_dict[hcc_feature+'_most_freq_count']
    return result_dict

def mult_type_hcc_feature_stat(recent_login_table,recent_trade_table):
    hcc_feature_list = ['ip','device','city']
    result_dict = {}
    for feature in hcc_feature_list:
        temp_result = build_hcc_stats_feature(recent_login_table,recent_trade_table,feature)
        result_dict=dict(result_dict, **temp_result)
    return result_dict

In [7]:
# for testing
test_row = trade_tt.iloc[111133]
recent_login_table = find_related_recent_logins_within_days(test_row,login_tt_has_trade,360)
recent_trade_table = find_related_recent_trades_within_days(test_row,trade_tt,360)
mult_type_hcc_feature_stat(recent_login_table,recent_trade_table)

{'city_freq_rank': 2.0,
 'city_max_min_dist': 0.90909090909090906,
 'city_most_freq_count': 11,
 'city_most_freq_rate': 0.5,
 'city_repeat_count': 10,
 'city_repeat_rate': 0.45454545454545453,
 'city_used_count': 3,
 'device_freq_rank': 4.0,
 'device_max_min_dist': 0.8571428571428571,
 'device_most_freq_count': 7,
 'device_most_freq_rate': 0.31818181818181818,
 'device_repeat_count': 5,
 'device_repeat_rate': 0.22727272727272727,
 'device_used_count': 6,
 'ip_freq_rank': 9.0,
 'ip_max_min_dist': 0.75,
 'ip_most_freq_count': 4,
 'ip_most_freq_rate': 0.18181818181818182,
 'ip_repeat_count': 2,
 'ip_repeat_rate': 0.09090909090909091,
 'ip_used_count': 14}

In [9]:
"""
works on spark
"""
import pyspark as spark
date_range_list = [360,30]

conf = spark.SparkConf().setAppName('jupyter_backend').setMaster('local[15]')\
        .set('spark.executor.memory','2g')\
        .set('spark.default.parallelism','112')
sc = spark.SparkContext(conf=conf)

result_df = get_transfer_result_wihtin_days(trade_tt,login_tt_has_trade,date_range_list,sc,mult_type_hcc_feature_stat)

In [14]:
result_df.columns

Index(['city_freq_rank_30_t_t', 'city_freq_rank_360_t_t',
       'city_max_min_dist_30_t_t', 'city_max_min_dist_360_t_t',
       'city_most_freq_count_30_t_t', 'city_most_freq_count_360_t_t',
       'city_most_freq_rate_30_t_t', 'city_most_freq_rate_360_t_t',
       'city_repeat_count_30_t_t', 'city_repeat_count_360_t_t',
       'city_repeat_rate_30_t_t', 'city_repeat_rate_360_t_t',
       'city_used_count_30_t_t', 'city_used_count_360_t_t',
       'device_freq_rank_30_t_t', 'device_freq_rank_360_t_t',
       'device_max_min_dist_30_t_t', 'device_max_min_dist_360_t_t',
       'device_most_freq_count_30_t_t', 'device_most_freq_count_360_t_t',
       'device_most_freq_rate_30_t_t', 'device_most_freq_rate_360_t_t',
       'device_repeat_count_30_t_t', 'device_repeat_count_360_t_t',
       'device_repeat_rate_30_t_t', 'device_repeat_rate_360_t_t',
       'device_used_count_30_t_t', 'device_used_count_360_t_t',
       'ip_freq_rank_30_t_t', 'ip_freq_rank_360_t_t', 'ip_max_min_dist_30_t_t',


In [15]:
#result_df.columns = [str(col) + '_t' for col in result_df.columns]
#del result_df['rowkey_t']
#del result_df['ori_id_t']

features = list(result_df.columns)
result_df = result_df.fillna(-10)
check_nan_inf(result_df,features)

In [17]:
for feature in features:
    pd.to_pickle(result_df[feature].values,feature_path+feature+'.pkl')
config = pd.read_pickle('config.pkl')
config.feature_dict['hcc_user_trade_habbit'] = features

In [30]:
config.feature_dict.keys()

dict_keys(['trade_and_recent_login_comparing', 'recent_login_detail', 'trade_detail_feature', 'login_trade_hist_stats', 'llc_user_habbit', 'hcc_user_habbit', 'hcc_properties', 'hcc_target_encoding', 'login_detail_new_features', 'hcc_mult_target_encoding', 'hcc_user_trade_habbit'])

### Getting the hcc properties

In [18]:
def build_hcc_property_dict(recent_logins,hcc,original_trade_id):
    result_dict = {}
    
    #hcc_trade_count = recent_trades.shape[0]
    hcc_login_count = recent_logins.shape[0]
    
    result_dict[hcc+'_login_count'] = hcc_login_count
    result_dict[hcc+'_same_id_login_count'] = recent_logins[recent_logins['id'] == original_trade_id].shape[0]
    
    hcc_related_id_series =  recent_logins.groupby('id').size()
    
    result_dict[hcc+'_diff_id_counts'] = len(hcc_related_id_series)
    
    if hcc_login_count ==0:
        result_dict[hcc+'_most_used_id_counts'] = 0
        result_dict[hcc+'_same_id_login_rate'] = 0
        result_dict[hcc+'_most_used_id_rate'] = 0
    else:
        result_dict[hcc+'_most_used_id_counts'] = hcc_related_id_series.max()
        result_dict[hcc+'_same_id_login_rate'] = result_dict[hcc+'_same_id_login_count']*1.0/hcc_login_count
        result_dict[hcc+'_most_used_id_rate'] = result_dict[hcc+'_most_used_id_counts']*1.0/hcc_login_count
    return result_dict

In [19]:
trade_tt_with_recent_logins = pd.read_csv(data_path+'trade_tt_with_recent_3_log_id.csv')
trade_tt_with_recent_logins['time'] = trade_tt_with_recent_logins['time'].apply(lambda x : datetime.datetime.strptime(x , '%Y-%m-%d %H:%M:%S'))
del trade_tt_with_recent_logins['month']
del trade_tt_with_recent_logins['day']
del trade_tt_with_recent_logins['weekday']
del trade_tt_with_recent_logins['hour']
del trade_tt_with_recent_logins['day_cycle']
del trade_tt_with_recent_logins['weekday_cycle']
del trade_tt_with_recent_logins['hour_cycle']
del trade_tt_with_recent_logins['recent_login_id_1']
del trade_tt_with_recent_logins['recent_login_id_2']
trade_wit_last_login_tt = trade_tt_with_recent_logins.merge(login_tt, how = 'left',left_on ='recent_login_id_0',right_on = 'log_id',suffixes=('', '_login'))

In [20]:
"""
works on spark
"""
import pyspark as spark
date_range_list = [360,30]

conf = spark.SparkConf().setAppName('jupyter_backend').setMaster('local[15]')\
        .set('spark.executor.memory','2g')\
        .set('spark.default.parallelism','112')
#sc = spark.SparkContext(conf=conf)

In [21]:
#parallel computing
trade_tt_rdd_buffer = []
for (idx,row) in trade_wit_last_login_tt.iterrows():
    trade_tt_rdd_buffer.append((idx,row))
trade_tt_rdd = sc.parallelize(trade_tt_rdd_buffer)

result_rdd = trade_tt_rdd.map(lambda x : mult_type_hcc_feature(x,login_tt_has_trade,build_hcc_property_dict,date_range_list))
result_rdd_buffer = result_rdd.collect()

In [22]:
#unfolding the result
result_rdd_to_df_buffer = []
for ori_id,rowkey,result_dict in result_rdd_buffer:
    unit_dict= {}
    unit_dict['ori_id'] = ori_id
    unit_dict['rowkey'] = rowkey
    
    for date_range in date_range_list:
        for key in result_dict[date_range]:
            unit_dict[key+'_'+str(date_range)]= result_dict[date_range][key]
    
    result_rdd_to_df_buffer.append(unit_dict)

result_df = pd.DataFrame(result_rdd_to_df_buffer)

assert (np.sum(result_df['rowkey']!=trade_tt['rowkey']))==0
assert (np.sum(result_df['ori_id']!=trade_tt.index))==0

In [26]:
#result_df.columns = [str(col) + '_t' for col in result_df.columns]
#del result_df['rowkey_t']
#del result_df['ori_id_t']

features = list(result_df.columns)
result_df = result_df.fillna(-10)
check_nan_inf(result_df,features)

In [28]:
check_nan_inf(result_df,features)

In [29]:
print(features)

['device_diff_id_counts_30_t', 'device_diff_id_counts_360_t', 'device_login_count_30_t', 'device_login_count_360_t', 'device_most_used_id_counts_30_t', 'device_most_used_id_counts_360_t', 'device_most_used_id_rate_30_t', 'device_most_used_id_rate_360_t', 'device_same_id_login_count_30_t', 'device_same_id_login_count_360_t', 'device_same_id_login_rate_30_t', 'device_same_id_login_rate_360_t', 'ip_diff_id_counts_30_t', 'ip_diff_id_counts_360_t', 'ip_login_count_30_t', 'ip_login_count_360_t', 'ip_most_used_id_counts_30_t', 'ip_most_used_id_counts_360_t', 'ip_most_used_id_rate_30_t', 'ip_most_used_id_rate_360_t', 'ip_same_id_login_count_30_t', 'ip_same_id_login_count_360_t', 'ip_same_id_login_rate_30_t', 'ip_same_id_login_rate_360_t']


In [31]:
for feature in features:
    pd.to_pickle(result_df[feature].values,feature_path+feature+'.pkl')
#config = pd.read_pickle('config.pkl')
config.feature_dict['hcc_trade_properties'] = features

In [36]:
pd.to_pickle(config,'config.pkl')

In [35]:
config.feature_dict

{'hcc_mult_target_encoding': ['ip_device_t_encoding',
  'ip_id_t_encoding',
  'device_id_t_encoding',
  'city_id_t_encoding',
  'device_id_city_t_encoding'],
 'hcc_properties': ['device_diff_id_counts_30',
  'device_diff_id_counts_360',
  'device_diff_id_counts_7',
  'device_login_count_30',
  'device_login_count_360',
  'device_login_count_7',
  'device_most_used_id_counts_30',
  'device_most_used_id_counts_360',
  'device_most_used_id_counts_7',
  'device_most_used_id_rate_30',
  'device_most_used_id_rate_360',
  'device_most_used_id_rate_7',
  'device_same_id_login_count_30',
  'device_same_id_login_count_360',
  'device_same_id_login_count_7',
  'device_same_id_login_rate_30',
  'device_same_id_login_rate_360',
  'device_same_id_login_rate_7',
  'ip_diff_id_counts_30',
  'ip_diff_id_counts_360',
  'ip_diff_id_counts_7',
  'ip_login_count_30',
  'ip_login_count_360',
  'ip_login_count_7',
  'ip_most_used_id_counts_30',
  'ip_most_used_id_counts_360',
  'ip_most_used_id_counts_7',
  